In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import json 
import sys
import os 

##Scope: 2.	Es “éxito” el que tu video aparezca más días como trending?


In [2]:
# Read original dataset. Date range varies from Nov 2017 up to June 2018
us_csv = "Resources/large_USvideos.csv"

original_df = pd.read_csv(us_csv)
original_df.head(500)

,version https://git-lfs.github.com/spec/v1
0,oid sha256:b05a31bb6bceef821b3c00da5e7d1771651...
1,size 64135657


In [3]:
original_df.set_index("video_id")
original_df.loc['1ZAPwfrtAFY']

KeyError: 'video_id'

In [ ]:
# Rename column name 
df= original_df.rename(columns={"video_id": "id"})

#                                        "trending_date": "Trending Date",
#                                        "title": "Title",
#                                        "channel_title": "Channel",
#                                        "category_id": "Category",
#                                        "publish_time": "Publish Date",
#                                        "tags": "Tags",
#                                        "views": "Views",
#                                        "likes": "Likes",
#                                        "dislikes": "Dislikes",
#                                        "comment_count": "Comments"})

df.columns

## Cleaning Data 

In [ ]:
# Convert column Title to propper case  
df["title"] = df.title.str.capitalize()

# Deleting columns with no info 
del df['ratings_disabled']
del df['comments_disabled']
del df['video_error_or_removed']


In [ ]:
df.head(100)

In [ ]:
df["id"].count()

In [ ]:
# Count how many days each video was trending 
video_by_id = df.set_index("id").groupby(["id"])
video_trending_count = video_by_id.count()["title"]

video_trending_count.sort_values(ascending=False)

### 2. Es Exito =  el que tu video aparezca más días como trending?

In [ ]:
## Cual es la categoria que tiene mas videos trending 

df_top_freq = df.groupby(['id', 'category_id'])['id'].agg(
    {"video_count": len}).sort_values(
    "video_count", ascending=False).head(100).reset_index()
df_top_freq.head()

In [ ]:
# que categoria se repite mas veces? 
category_frecuence = df_top_freq.groupby("category_id").id.nunique()
category_frecuence.sort_values(ascending=False)
category_frecuence_df = pd.DataFrame(category_frecuence) 
category_frecuence_df.sort_values("id",ascending=False)
category_frecuence_df= category_frecuence_df.rename(columns={"id": "category_count"})
category_frecuence_df.sort_values("category_count",ascending=False)

In [ ]:
#pasar categorias a nombres 
filepath = os.path.join("Resources/US_category_id.json")
with open(filepath) as jsonfile:
    categories_data = json.load(jsonfile)
categories_data


In [ ]:
category_frecuence_df["category_name"] = ""
category_frecuence_df.sort_values("category_count",ascending=False)
category_frecuence_df.reset_index()

In [ ]:
#puedo acer una definicion para que solo tome el numero del id y de el resto 
#!!!!!!!!!!!!!!!!!!!!! algo no esta bien pues la cat 19 es Travel & Events y me esta dando film and animation! 
# la def esta basada en el category_count' en vez del category_id pero cuando lo cambio me da problemas
# quizas es por el index ?  intente reset el index pero tampoco me funciona ??? 

def video_category_name(dF): 
    for index,row in category_frecuence_df.iterrows():
        x = category_frecuence_df.loc[index,'category_count']
        if category_frecuence_df.loc[index,'category_count'] == x: 
            category_frecuence_df.loc[index,"category_name"] = categories_data["items"][x-1]["snippet"]["title"] 
video_category_name(category_frecuence_df)
category_frecuence_df.sort_values("category_count",ascending=False)

In [ ]:
category_names = category_frecuence_df["category_name"]
category_names.head(50)

In [ ]:
category_frecuence_df.reset_index().head(100)


In [ ]:
category_names = category_frecuence_df["category_name"]
category_names

In [ ]:
category_count = category_frecuence_df["category_count"]
category_count

In [ ]:
#!!!!!!!!!!!!!!!!!!!!! necesito decirle que ponga los labels verticales y ordenarlo de mayor a menor quizas? ojo 
# si lo quiero ordenar quizas se pierda el orden de los elementos ojo pues fueron sacados de listas separadas y no de un DF

plt.bar(category_names, category_count, facecolor="red", alpha=0.75, align="center")
plt.xticks(rotation=90)

In [ ]:
# bar plot eje x categoria eje y len category_frecuence_df
#!!!!!!!!!!!!!!!!!!!!!!!!!! falta sacar nombre e la categoria/ 

# Create a bar chart based off of the group series from before
category_frecuence_chart = category_frecuence_df.plot(kind='bar')

# Set the xlabel and ylabel using class methods
category_frecuence_chart.set_xlabel("Category Name")
category_frecuence_chart.set_ylabel("Qty of Trending Videos")


plt.show()
plt.tight_layout()

## top 100 videos - views

In [ ]:
# Cuantos views obtuvo cada video de los 100 mas trending 
# ojo. de la lista de 100 trending videos, necesito la cantidad maxima de views 

#top100videos_views = df.groupby(['id', 'views']).max()
#top100videos_views.head()


video_by_id = df.set_index("id").groupby(["id"])
top100videos_views = video_by_id.max()["views"]

top100videos_views_df = pd.DataFrame(top100videos_views)
#top100videos_views_df.sort_values("views",ascending=False)

top100videos_views_df = top100videos_views_df.groupby(['id']).head(100)
top100videos_views_df["views"] = top100videos_views_df["views"]
top100videos_views_df.sort_values("views",ascending=False)

In [ ]:
top100videos_views_chart = top100videos_views_df.plot(kind='bar')

# Set the xlabel and ylabel using class methods
top100videos_views_chart.set_xlabel("Video Id")
top100videos_views_chart.set_ylabel("Max Views")


plt.show()
plt.tight_layout()